# 📚 建立 Stable Diffusion 的圖片生成 Web App

### 1. 安裝必要套件

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

### 輸入 HuggingFace token 金鑰

In [ ]:
from huggingface_hub import login

from google.colab import userdata

hf_token = userdata.get("HuggingFace")
login(token=hf_token)

In [ ]:
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import torch
import gc
import matplotlib.pyplot as plt
import gradio as gr
import random

### 2. 指定並讀入模型
### 🧠 模型介紹：gsdf/Counterfeit-V2.5

`Counterfeit-V2.5` 是一款基於 Stable Diffusion 1.5 的強化插畫風格模型，特別適合生成 **動漫風格、半寫實風格** 以及 **唯美角色圖像**。此模型由社群創作者精心微調，融合了高質感人物特徵、細膩背景紋理與柔和打光技術，廣泛應用於角色設計、同人創作與幻想場景生成。

- ✅ 支援 **diffusers** 格式，適用於 Hugging Face 生態系。
- 🎨 擅長生成：精緻五官、光影豐富的臉部、複雜髮絲、動漫人物、幻想背景。
- 💡 推薦搭配：high-resolution、perfect face、sharp lines 等增強詞使用。
- 📌 模型來源：[gsdf/Counterfeit-V2.5 on Hugging Face](https://huggingface.co/gsdf/Counterfeit-V2.5)



In [ ]:
model_name = "gsdf/Counterfeit-V2.5"

注意有可能要停用 `use_safetensors=True`。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

In [ ]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

### 🛠️ 3. 生成的 `generate_images` 函式

本系統使用自定義的 `generate_images()` 函式進行圖像生成，支援多張圖片、隨機種子、自動加強提示詞與負面詞等功能，搭配 `gsdf/Counterfeit-V2.5` 模型，可靈活控制生成風格與品質。

### 📌 函式功能包含：

- ✅ 支援 **多張圖片生成**（指定張數，每張使用不同種子）
- ✅ 可選擇使用 **自訂 Random Seed** 或自動隨機產生
- ✅ 自動組合使用者輸入的 Prompt 與 Enhance Text
- ✅ 可勾選是否使用 Negative Prompt，提升畫面品質
- ✅ 自動釋放 GPU 記憶體，減少長時間生成造成的記憶體問題
- ✅ 若高度或寬度不是 8 的倍數，會主動報錯提醒（符合 Stable Diffusion 限制）

### 🧠 主要參數說明：

| 參數名稱        | 說明 |
|------------------|------|
| `prompt`         | 使用者輸入的主提示詞，建議使用英文 |
| `enhance_text`   | 高品質增強詞，例如：masterpiece、ultra-detailed 等 |
| `negative_text`  | 排除元素的提示詞，避免錯誤結構或低畫質內容 |
| `height/width`   | 輸出影像尺寸，需為 8 的倍數（如：512、768）|
| `steps`          | 推理步數，越多越精緻但花費時間越長 |
| `num_images`     | 一次生成幾張圖片 |
| `seed`           | 可固定隨機種子，以利重現相同結果 |


In [ ]:
def generate_images(prompt, use_enhance, enhance_text, use_negative, negative_text,
                    use_custom_seed, custom_seed, height, width, steps, num_images):

    height = int(height)
    width = int(width)

    if height % 8 != 0 or width % 8 != 0:
        raise ValueError("高度和寬度必須是8的倍數！")

    if use_custom_seed:
        base_seed = int(custom_seed)
    else:
        base_seed = random.randint(0, 2**32 - 1)

    seeds = [base_seed + i for i in range(num_images)]

    prompts = []
    negative_prompts = []
    generators = []

    final_prompt = prompt
    if use_enhance and enhance_text:
        final_prompt = prompt + ", " + enhance_text

    final_negative = negative_text if use_negative else None

    for seed in seeds:
        g = torch.Generator("cuda").manual_seed(seed)
        generators.append(g)
        prompts.append(final_prompt)
        negative_prompts.append(final_negative)

    gc.collect()
    torch.cuda.empty_cache()

    images = []
    for i in range(num_images):
        with torch.no_grad():
            image = pipe(
                prompt=prompts[i],
                negative_prompt=negative_prompts[i] if final_negative else None,
                height=height,
                width=width,
                num_inference_steps=steps,
                guidance_scale=7.5,
                generator=generators[i]
            ).images[0]
            images.append(image)

    return images, f"使用的 random seeds: {seeds}"

### 4. 打造 Gradio Web App

這個生成器使用 `gsdf/Counterfeit-V2.5` 模型，可輸入提示詞，快速產出細膩的插畫風圖片。可以：

- **輸入提示詞（Prompt）**：建議使用英文描述，例如 `1girl, silver hair, red eyes, fantasy forest`
- **選擇是否自動加強**：內建的加強內容會補充關鍵詞，讓圖片品質更高（如高解析、精緻光影）
- **啟用 Negative Prompt**：排除你不想要出現的內容，例如手部錯誤、低解析、浮水印等
- **可選擇自訂隨機種子（Seed）**：若想重現相同圖像，可固定種子值
- **調整圖片尺寸與生成張數**：高度與寬度需為 8 的倍數，否則會報錯提醒

所有設定會送入 `generate_images()` 函式中，由 Stable Diffusion 模型進行處理，並依據你的指示完成圖像生成！


In [ ]:
default_enhance = "masterpiece, best quality, illustration, ultra-detailed, high-resolution, sharp lines, perfect face, normal eye count, detailed eyes, beautiful skin, vibrant lighting, dynamic pose, anime style, highly detailed background"
default_negative = "lowres, blurry, bad anatomy, bad hands, fused fingers, extra fingers, missing fingers, long neck, deformed face, duplicate limbs, text, watermark, signature, poorly drawn, ugly eyes, morphed body"

# Gradio UI
with gr.Blocks(css="""
.gradio-container {
    background-color: #1E1E2F;
    color: #FFFFFF;
    padding: 20px;
}
.gr-button {
    font-size: 18px;
    background: linear-gradient(to right, #8A2BE2, #4B0082);
    color: white;
}
.gr-textbox textarea, .gr-number input, .gr-dropdown select {
    background-color: #2C2C3C;
    color: #FFFFFF;
    border: 1px solid #555;
}
.gr-slider .wrap {
    background-color: #2C2C3C;
}
""") as demo:

    gr.Markdown("""
    # 🎨 Counterfeit V2.5 插畫生成器
    ✨ 請直接輸入提示詞（建議使用英文），可搭配自訂強化與負面詞，快速生成高品質插畫風圖像！
    💡 模型：gsdf/Counterfeit-V2.5（支援 diffusers，擅長動漫 × 插畫 × 半寫實風格）
    """)

    with gr.Row():
        with gr.Column(scale=6):
            prompt = gr.Textbox(label="📝 提示詞 Prompt", placeholder="例如：1girl, silver hair, red eyes, fantasy forest", lines=3)
            with gr.Row():
                use_enhance = gr.Checkbox(label="✅ 自動加強 Prompt", value=True)
                enhance_text = gr.Textbox(label="加強內容（可自訂）", value=default_enhance)
            with gr.Row():
                use_negative = gr.Checkbox(label="🚫 使用 Negative Prompt", value=True)
                negative_text = gr.Textbox(label="Negative Prompt 內容（可自訂）", value=default_negative)
            with gr.Row():
                use_custom_seed = gr.Checkbox(label="🎲 自訂 Random Seed", value=False)
                custom_seed = gr.Number(label="指定 seed（選填）", value=42)
            with gr.Row():
                height = gr.Dropdown(["512", "768", "1024"], label="🖼️ 高度 Height", value="512")
                width = gr.Dropdown(["512", "768", "1024"], label="📐 寬度 Width", value="512")
            with gr.Row():
                steps = gr.Slider(10, 50, value=20, step=5, label="🧠 生成步數 (Steps)")
                num_images = gr.Slider(1, 4, step=1, value=1, label="🖼️ 生成張數")
            generate_btn = gr.Button("🚀 生成插畫")

        with gr.Column(scale=6):
            gallery = gr.Gallery(label="🌈 生成結果", columns=2, object_fit="contain", height="auto")
            seed_info = gr.Label(label="📦 使用的 Seed")

    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, use_enhance, enhance_text, use_negative, negative_text,
                use_custom_seed, custom_seed, height, width, steps, num_images],
        outputs=[gallery, seed_info]
    )



In [ ]:
demo.launch(share=True, debug=True)